In [2]:
import tensorflow
from tensorflow.keras.models import load_model

from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle
import pandas as pd
import numpy as np

In [16]:
#Load Model
model = load_model('model.h5')

# Loading Standardscaler,Label_encoder,One_hot_encoder
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [13]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [17]:
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [18]:
# Gender Transformation
input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])

In [19]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [21]:
# Geography transformation
geo = one_hot_encoder_geo.transform(input_data_df[['Geography']]).toarray()
geo_df = pd.DataFrame(geo,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [22]:
input_data_df = pd.concat([input_data_df.drop(columns='Geography',axis=1),geo_df],axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [24]:
# Scaling encoded data
input_scaled = scaler.transform(input_data_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [25]:
prediction = model.predict(input_scaled)

1/1 [==============================] - 0s 343ms/step


In [26]:
prediction

array([[0.04202544]], dtype=float32)

In [27]:
prediction_prob = prediction[0][0]

In [28]:
prediction_prob

0.042025436

In [29]:
if prediction_prob > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
